# Results — offering_no_network

This notebook summarizes generated datasets, instances, and (if available) trained model / evaluation artifacts for **`offering_no_network`** in the Neur2RO codebase.

It is written to be **robust**:
- Works even if some result files are missing (it will skip those sections).
- Supports MATLAB `.mat` files saved in **v7.3** (HDF5) via `h5py` fallback.


In [ ]:
# --- Imports / setup ---
import os, glob, pickle, math, re
from dataclasses import dataclass
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Optional dependencies
try:
    import h5py  # for MATLAB v7.3 .mat
except Exception:
    h5py = None

# Make plots render a bit larger by default
plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True

# Project root (adjust if needed)
PROJECT_ROOT = os.getcwd()

DATA_ROOT = os.path.join(PROJECT_ROOT, "data")
PROB_NAME  = "offering_no_network"
PROB_DIR   = os.path.join(DATA_ROOT, PROB_NAME)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("PROB_DIR:", PROB_DIR)


In [ ]:
# --- Helpers ---
def latest_file(pattern: str):
    """Return latest file by mtime matching a glob pattern, or None."""
    files = glob.glob(pattern)
    if not files:
        return None
    files.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    return files[0]

def load_pkl(path: str):
    with open(path, "rb") as f:
        return pickle.load(f)

def try_load_price_matrix(mat_path: str, var_candidates=("price_matrix_revised_100", "price_matrix")):
    """Load a 24xS price matrix from a .mat. Supports v7.3 via h5py fallback."""
    # 1) scipy.io.loadmat (MATLAB v5/v7 non-HDF)
    try:
        from scipy.io import loadmat
        try:
            mat = loadmat(mat_path)
            for var in var_candidates:
                if var in mat:
                    arr = np.asarray(mat[var], dtype=float)
                    return arr
            # fallback: any 2D 24xS
            for k, v in mat.items():
                if k.startswith("__"):
                    continue
                if isinstance(v, np.ndarray) and v.ndim == 2 and v.shape[0] == 24:
                    return np.asarray(v, dtype=float)
            raise KeyError(f"No variable in {var_candidates} and no 24xS array found.")
        except NotImplementedError:
            # MATLAB v7.3 -> fall through to h5py
            pass
    except Exception:
        pass

    # 2) h5py fallback (MATLAB v7.3)
    if h5py is None:
        raise RuntimeError("MATLAB file looks like v7.3 (HDF5), but h5py is not available.")

    with h5py.File(mat_path, "r") as f:
        # Try candidate names
        for var in var_candidates:
            if var in f:
                arr = np.array(f[var])
                break
        else:
            # try to find a dataset with one dimension=24
            arr = None
            def _visit(name, obj):
                nonlocal arr
                if arr is not None:
                    return
                if hasattr(obj, "shape") and len(obj.shape) == 2 and (24 in obj.shape):
                    arr = np.array(obj)
            f.visititems(_visit)
            if arr is None:
                raise KeyError("No 2D dataset with a dimension of 24 found in the v7.3 file.")

        # MATLAB stores arrays column-major; h5py often yields transposed orientation.
        # We want 24 x S
        if arr.shape[0] != 24 and arr.shape[1] == 24:
            arr = arr.T
        if arr.shape[0] != 24:
            raise ValueError(f"Expected 24xS, got {arr.shape}")
        return arr.astype(float)

def safe_makedirs(path: str):
    os.makedirs(path, exist_ok=True)


In [ ]:
# --- Load config (if available) ---
cfg = None
try:
    import ro.params as ro_params
    cfg = getattr(ro_params, PROB_NAME)
    print("Loaded cfg from ro.params:", cfg)
except Exception as e:
    print("Could not import ro.params / cfg. Using fallback paths only.")
    print("Reason:", repr(e))

# If cfg exists, prefer its data_path (some repos use ./data/, some use data/)
if cfg is not None and hasattr(cfg, "data_path"):
    # normalize to PROJECT_ROOT
    cfg_data_path = cfg.data_path
    # if cfg.data_path is relative, interpret under PROJECT_ROOT
    DATA_ROOT = cfg_data_path if os.path.isabs(cfg_data_path) else os.path.join(PROJECT_ROOT, cfg_data_path)
    PROB_DIR = os.path.join(DATA_ROOT, PROB_NAME)

print("Resolved DATA_ROOT:", DATA_ROOT)
print("Resolved PROB_DIR:", PROB_DIR)

safe_makedirs(PROB_DIR)


In [ ]:
# --- Load generated ML dataset (ml_data*.pkl) ---
# Typical filenames:
#   ./data/offering_no_network/ml_data.pkl
#   ./data/offering_no_network/ml_data_YYYYMMDD....pkl  (if you used --ml_suffix)
ml_data_path = latest_file(os.path.join(PROB_DIR, "ml_data*.pkl"))
print("Latest ml_data file:", ml_data_path)

if ml_data_path is None:
    raise FileNotFoundError(
        f"No ml_data*.pkl found under {PROB_DIR}. "
        "Run: python -m ro.scripts.02_generate_dataset --problem offering_no_network ..."
    )

dataset = load_pkl(ml_data_path)

# Expected: dict with keys like tr_data, val_data, maybe other metadata
print("Dataset keys:", list(dataset.keys()))
print("Train samples:", len(dataset.get("tr_data", [])))
print("Val samples:", len(dataset.get("val_data", [])))


In [ ]:
# --- Convert samples to a DataFrame for summary ---
def flatten_samples(samples, split_name):
    rows = []
    for r in samples:
        # DataManager stores these fields in result dict
        inst = r.get("instance", {})
        rho = float(inst.get("rho", 1.0))
        fs = float(r.get("fs_obj", np.nan))
        ss = float(r.get("ss_obj", np.nan))
        # Two common conventions:
        total_1 = fs + ss          # unweighted sum
        total_2 = fs + rho * ss    # probability-weighted second-stage cost (matches Matlab MP scaling)
        row = {
            "split": split_name,
            "scenario_id": int(inst.get("scenario_id", -1)),
            "rho": rho,
            "fs_obj": fs,
            "ss_obj": ss,
            "total_fs_plus_ss": total_1,
            "total_fs_plus_rho_ss": total_2,
            "time_s": float(r.get("time", np.nan)),
        }
        rows.append(row)
    return rows

rows = []
rows += flatten_samples(dataset.get("tr_data", []), "train")
rows += flatten_samples(dataset.get("val_data", []), "val")
df = pd.DataFrame(rows)

df.head()


In [ ]:
# --- Summary stats ---
def summarize(col):
    s = df[col].dropna()
    return pd.Series({
        "count": int(s.shape[0]),
        "mean": float(s.mean()) if len(s) else np.nan,
        "std": float(s.std(ddof=1)) if len(s) > 1 else np.nan,
        "min": float(s.min()) if len(s) else np.nan,
        "p25": float(s.quantile(0.25)) if len(s) else np.nan,
        "median": float(s.median()) if len(s) else np.nan,
        "p75": float(s.quantile(0.75)) if len(s) else np.nan,
        "max": float(s.max()) if len(s) else np.nan,
    })

summary = pd.DataFrame({
    "fs_obj": summarize("fs_obj"),
    "ss_obj": summarize("ss_obj"),
    "total_fs_plus_ss": summarize("total_fs_plus_ss"),
    "total_fs_plus_rho_ss": summarize("total_fs_plus_rho_ss"),
    "time_s": summarize("time_s"),
}).T

summary


In [ ]:
# --- Per-scenario distribution (validation split) ---
df_val = df[df["split"] == "val"].copy()
if df_val.empty:
    print("No validation samples; showing all samples.")
    df_val = df.copy()

by_s = df_val.groupby("scenario_id").agg(
    n=("scenario_id", "size"),
    mean_total=("total_fs_plus_rho_ss", "mean"),
    std_total=("total_fs_plus_rho_ss", "std"),
    mean_fs=("fs_obj", "mean"),
    mean_ss=("ss_obj", "mean"),
).reset_index().sort_values("scenario_id")

by_s.head(10)


In [ ]:
# --- Load and visualize the underlying instance profiles (price matrix, load, PV bounds) ---
# 1) Locate price .mat path
mat_path = None
if cfg is not None and hasattr(cfg, "price_mat_path"):
    mat_path = cfg.price_mat_path
    if not os.path.isabs(mat_path):
        mat_path = os.path.join(PROJECT_ROOT, mat_path)

if mat_path is None:
    # fallback: pick any .mat in problem dir
    mat_path = latest_file(os.path.join(PROB_DIR, "*.mat"))

print("Price matrix .mat path:", mat_path)

# 2) Load price matrix
price_matrix = None
if mat_path is not None and os.path.exists(mat_path):
    price_matrix = try_load_price_matrix(mat_path)
    print("price_matrix shape:", price_matrix.shape)
else:
    print("No .mat price file found; skipping price matrix plots.")

# 3) Load one instance from dataset to pull load/PV bounds
sample_any = None
for split in ("val", "train"):
    data = dataset.get(f"{split}_data", [])
    if data:
        sample_any = data[0]
        break

if sample_any is None:
    raise RuntimeError("Dataset is empty.")

inst = sample_any["instance"]
T = int(inst.get("T", 24))
load = np.asarray(inst["p_load"], dtype=float).reshape(-1)
pv_min = np.asarray(inst["p_pv_min"], dtype=float).reshape(-1)
pv_max = np.asarray(inst["p_pv_max"], dtype=float).reshape(-1)
lam_rt = np.asarray(inst["lambda_rt"], dtype=float).reshape(-1)
Sbase = float(getattr(cfg, "Sbase", 1000.0)) if cfg is not None else 1000.0

# --- plots ---
t = np.arange(T)

fig, ax = plt.subplots()
ax.plot(t, load, label="load (pu)")
ax.plot(t, pv_min, label="pv_min (pu)")
ax.plot(t, pv_max, label="pv_max (pu)")
ax.set_title("Load and PV bounds (per-unit)")
ax.set_xlabel("Hour")
ax.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(t, lam_rt, label="lambda_rt")
ax.set_title("Real-time price (lambda_rt)")
ax.set_xlabel("Hour")
ax.legend()
plt.show()

if price_matrix is not None:
    fig, ax = plt.subplots()
    im = ax.imshow(price_matrix, aspect="auto")
    ax.set_title("Day-ahead price matrix (24 x S)")
    ax.set_xlabel("Scenario")
    ax.set_ylabel("Hour")
    plt.colorbar(im, ax=ax, label="DA price (raw units from .mat)")
    plt.show()


In [ ]:
# --- Objective distributions ---
cols = ["fs_obj", "ss_obj", "total_fs_plus_ss", "total_fs_plus_rho_ss"]
for col in cols:
    vals = df_val[col].dropna().values
    if len(vals) == 0:
        print(f"Skip {col}: no values")
        continue
    fig, ax = plt.subplots()
    ax.hist(vals, bins=30)
    ax.set_title(f"Validation distribution: {col}")
    ax.set_xlabel(col)
    ax.set_ylabel("count")
    plt.show()

# Boxplot across scenarios for total (validation)
if df_val["scenario_id"].nunique() > 1:
    # keep scenarios ordered
    scens = sorted(df_val["scenario_id"].unique().tolist())
    data = [df_val[df_val["scenario_id"]==s]["total_fs_plus_rho_ss"].dropna().values for s in scens]
    fig, ax = plt.subplots(figsize=(12,4))
    ax.boxplot(data, labels=scens, showfliers=False)
    ax.set_title("Validation total_fs_plus_rho_ss by scenario")
    ax.set_xlabel("scenario_id")
    ax.set_ylabel("total_fs_plus_rho_ss")
    plt.xticks(rotation=0)
    plt.show()


In [ ]:
# --- Inspect one sample and (optionally) re-solve second-stage to visualize dispatch ---
# This section needs gurobipy (and a working license). If unavailable, it will be skipped.

sample = df_val.sample(1, random_state=0).iloc[0] if not df_val.empty else df.sample(1, random_state=0).iloc[0]
# Find the actual record dict that matches this row (best-effort)
def find_record(split, scenario_id):
    for r in dataset.get(f"{split}_data", []):
        inst = r.get("instance", {})
        if int(inst.get("scenario_id", -999)) == int(scenario_id):
            return r
    return None

rec = find_record("val", int(sample["scenario_id"])) or find_record("train", int(sample["scenario_id"]))
if rec is None:
    print("Could not find matching record; skipping solve/plot.")
else:
    x  = np.asarray(rec["x"], dtype=float).reshape(-1)
    xi = np.asarray(rec["xi"], dtype=float).reshape(-1)
    inst = rec["instance"]

    print("scenario_id:", inst.get("scenario_id"))
    print("fs_obj:", rec.get("fs_obj"), "ss_obj:", rec.get("ss_obj"))
    print("x range:", float(x.min()), float(x.max()))
    print("xi range:", float(xi.min()), float(xi.max()))

    # Try to solve with the OfferingNoNetwork class if available
    try:
        from ro.two_ro.offering_no_network import OfferingNoNetwork
        two_ro = OfferingNoNetwork()
        fs_obj, ss_obj, model = two_ro.solve_second_stage(x, xi, inst, verbose=0)
        print("Re-solved: fs_obj=", fs_obj, "ss_obj=", ss_obj)

        # Extract trajectories
        T = int(inst.get("T", 24))
        p_pv   = np.array([model._p_pv[t].X for t in range(T)])
        p_ch   = np.array([model._p_ch[t].X for t in range(T)])
        p_dis  = np.array([model._p_dis[t].X for t in range(T)])
        soc    = np.array([model._soc[t].X for t in range(T)])
        p_buy  = np.array([model._p_rt_b[t].X for t in range(T)])
        p_sell = np.array([model._p_rt_s[t].X for t in range(T)])

        t = np.arange(T)

        fig, ax = plt.subplots()
        ax.plot(t, x, label="p_DA (x)")
        ax.plot(t, p_pv, label="p_pv")
        ax.plot(t, p_dis - p_ch, label="p_dis - p_ch")
        ax.set_title("DA schedule and realized PV/ESS")
        ax.set_xlabel("Hour")
        ax.legend()
        plt.show()

        fig, ax = plt.subplots()
        ax.plot(t, soc, label="SOC")
        ax.set_title("ESS State of Charge")
        ax.set_xlabel("Hour")
        ax.legend()
        plt.show()

        fig, ax = plt.subplots()
        ax.plot(t, p_buy, label="RT buy")
        ax.plot(t, p_sell, label="RT sell")
        ax.set_title("RT deviation trades")
        ax.set_xlabel("Hour")
        ax.legend()
        plt.show()

    except Exception as e:
        print("Skipping re-solve/plot due to error:", repr(e))


In [ ]:
# --- If training artifacts exist: plot training curves from random_search ---
# This looks for *tr_res*.pkl files produced by ro/scripts/03_train_model.py.

rs_dir = os.path.join(PROB_DIR, "random_search")
tr_res_path = latest_file(os.path.join(rs_dir, "*_tr_res*.pkl"))
print("Latest training results:", tr_res_path)

if tr_res_path is None:
    print("No training-results pickle found under:", rs_dir)
else:
    tr_res = load_pkl(tr_res_path)
    print("Training result keys:", list(tr_res.keys()))
    print("Best val_mae:", tr_res.get("val_mae"))
    stats = tr_res.get("training_stats", {})

    # Plot losses if present
    losses = stats.get("losses", None)
    if losses is not None and len(losses) > 0:
        fig, ax = plt.subplots()
        ax.plot(np.arange(1, len(losses)+1), losses)
        ax.set_title("Training loss (mean per epoch)")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        plt.show()

    # Plot validation MAE trajectory if present
    val_maes = stats.get("val_maes", None)
    if val_maes is not None and len(val_maes) > 0:
        fig, ax = plt.subplots()
        ax.plot(np.arange(1, len(val_maes)+1), val_maes)
        ax.set_title("Validation MAE over evaluations")
        ax.set_xlabel("Eval step")
        ax.set_ylabel("MAE")
        plt.show()


In [ ]:
# --- If evaluation artifacts exist: summarize eval_results/*.pkl (best-effort) ---
eval_dir = os.path.join(PROB_DIR, "eval_results")
eval_files = sorted(glob.glob(os.path.join(eval_dir, "*.pkl")))
print("Found eval_results files:", len(eval_files))

if not eval_files:
    print("No eval_results/*.pkl found. If you ran evaluation scripts, check this folder:", eval_dir)
else:
    rows = []
    for fp in eval_files:
        try:
            d = load_pkl(fp)
        except Exception:
            continue

        # Try common schemas used across Neur2RO notebooks
        row = {"file": os.path.basename(fp)}
        for k in ["method", "time", "obj", "objective", "gap", "feasible", "n_iters", "val_mae"]:
            if k in d:
                row[k] = d[k]
        # Sometimes results stored under nested keys
        if "results" in d and isinstance(d["results"], dict):
            for k in ["time", "obj", "gap", "feasible", "n_iters"]:
                if k in d["results"] and k not in row:
                    row[k] = d["results"][k]
        rows.append(row)

    df_eval = pd.DataFrame(rows)
    display(df_eval.head(20))
